In [ ]:
!pip install langchain_experimental
!pip install underthesea
!pip install langchain_huggingface

# Import library

In [6]:
# from PyPDF2 import PdfReader
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForQuestionAnswering, pipeline
import torch
import re
import os
import json
# import easyocr
# from ocr_tessa import fromPDFtoImg
device = torch.device('cuda:0') # if using cuda settings is different
# from verify import verify_qa
# from test_sentence import semantic_chunk
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_experimental.text_splitter import SemanticChunker
from sentence_transformers import SentenceTransformer
from langchain_huggingface import HuggingFaceEmbeddings
from underthesea import sent_tokenize


In [ ]:
# material_path = "ocr_material"
# pdf_name = "dialy12.pdf"
# file_name = os.path.splitext(pdf_name)[0]
# file_name = file_name.replace(' ', '_')
# print(file_name)
# pdf_file = os.path.join(material_path, pdf_name)
# output_file = f"qa_pairs_ocr/{file_name}/{file_name}_qa_pairs.json"
# open(output_file, 'w').close()

In [20]:
def create_output_folder(folder_path):
    """Tạo thư mục lưu trữ nếu chưa tồn tại."""
    os.makedirs(folder_path, exist_ok=True)
    print(f"Tạo '{folder_path}'")

def save_text_to_file(text, output_path):
    with open(output_path, "w", encoding="utf-8") as f:
        f.write(text)
    print(f"Lưu {output_path}")

# def clean_temp_folder(folder_path):
#     shutil.rmtree(folder_path)
#     print(f"Xóa '{folder_path}'")

In [24]:
material_path = "ocr_material"
pdf_name = "dialy.pdf"
txt_file_name = "dialy.txt"
file_name = os.path.splitext(pdf_name)[0]
file_name = file_name.replace(' ', '_')
print(file_name)
pdf_file = os.path.join(material_path, pdf_name)
txt_file = os.path.join(material_path, txt_file_name)

# output_file = f"qa_pairs_ocr/{file_name}/{file_name}_qa_pairs.json"
output_file = f"qa_pairs_ocr/{file_name}"
if not os.path.exists(output_file):
    create_output_folder(output_file)
# open(output_file, 'w').close()

dialy


In [25]:
model = HuggingFaceEmbeddings(model_name = 'dangvantuan/vietnamese-embedding')
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Initialize query model
query_model_name = "doc2query/msmarco-vietnamese-mt5-base-v1"
query_tokenizer = AutoTokenizer.from_pretrained(query_model_name)
query_model = AutoModelForSeq2SeqLM.from_pretrained(query_model_name)

# Initialize QA model
qa_tokenizer = AutoTokenizer.from_pretrained("ancs21/xlm-roberta-large-vi-qa")
qa_model = AutoModelForQuestionAnswering.from_pretrained("ancs21/xlm-roberta-large-vi-qa")
qa_pipeline = pipeline("question-answering", model=qa_model, tokenizer=qa_tokenizer, device=0) # CUDA

Device set to use cuda:0


In [26]:
# test_sentence
def semantic_chunk(text, max_len=300):
    sentences = text.split('. ')
    chunks, current_chunk = [], ""

    for sentence in sentences:
        if len(current_chunk) + len(sentence) <= max_len:
            current_chunk += sentence + ". "
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + ". "

    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

def chunk_to_sub_paragraph(text, chunk_size=300):
    l = sent_tokenize(text)
    buffer = ''
    chunk = []
    for i in range(len(l)):
        if (len(buffer) >= chunk_size):
            chunk.append(buffer)
            buffer = ''
        else:
            buffer += l[i]
    return chunk

def generate_queries(context):
    input_ids = query_tokenizer.encode(context, return_tensors='pt')
    with torch.no_grad():
        queries = query_model.generate(
            input_ids=input_ids,
            max_length=150,  # Adjust length
            num_beams=5,  # Increase beam size
            no_repeat_ngram_size=3,  # Avoid repetitions
            num_return_sequences=3,  # More queries for variety
        )
    return [query_tokenizer.decode(q, skip_special_tokens=True) for q in queries]

def generate_answer(question, context):
    result = qa_pipeline({"question": question, "context": context})
    return result['answer']

# def clean_text(text):
#     # Remove page numbers like "Trang 123" or "Page 123"
#     text = re.sub(r'\b(?:Trang)\s*\d+\b', '', text)
#     # Remove figures labeled "Hình:" or similar patterns
#     text = re.sub(r'\b(?:Hình|Hinh)\s*.*?(?=\n|$)', '', text)
#     # Remove extra whitespace and newlines
#     text = re.sub(r'\s+', ' ', text).strip()
#     return text

In [30]:
os.makedirs(os.path.join('qa_pairs_ocr', file_name), exist_ok=True)
question_existed = []
counter = 1
qa_pairs = []

# Process each page from the text file
with open(txt_file, "r", encoding="utf-8") as txt_reader:
    page_data = txt_reader.read()
    pages = re.split(r'Page \d+:', page_data)

    for i, page_content in enumerate(pages):
        if not page_content.strip():
            continue

        # text = clean_text(page_content)
        text = page_content
        sentences_list = re.split(r'(?<=[.!?])\s+', text.strip())
        if len(sentences_list) > 1:
            chunks = chunk_to_sub_paragraph(text)
        else:
            chunks = sentences_list

        # Log extracted text and chunks
        with open("log.txt", "a", encoding="utf-8") as f:
            f.write(f"All text is : {text}, and chunks is : {chunks}\n")

        # Generate QA pairs for each chunk
        for chunk in chunks:
            if not chunk.strip() or chunk.isnumeric():
                continue
            questions = generate_queries(chunk)
            questions_set = set(questions)

            for question in questions_set:
                counter += 1
                if not question.strip():
                    continue  # Skip empty questions
                if question not in question_existed:
                    answer = generate_answer(question, chunk)
                    # if verify_qa(chunk, question, answer):
                    qa_pairs.append({"id": f"{file_name}_p_{i + 1}_{counter}", "question": question, "answer": answer, "context": chunk})
                    question_existed.append(question)
		
		# check folder
        os.makedirs(os.path.join('qa_pairs_ocr', file_name), exist_ok=True)
        output_file = os.path.join('qa_pairs_ocr', file_name, f"{file_name}_qa_pairs.json")
        
        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(qa_pairs, f, ensure_ascii=False, indent=4)

        print(f"QA pairs saved to {output_file}")

QA pairs saved to qa_pairs_ocr/dialy/dialy_qa_pairs.json


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


QA pairs saved to qa_pairs_ocr/dialy/dialy_qa_pairs.json
QA pairs saved to qa_pairs_ocr/dialy/dialy_qa_pairs.json
QA pairs saved to qa_pairs_ocr/dialy/dialy_qa_pairs.json
QA pairs saved to qa_pairs_ocr/dialy/dialy_qa_pairs.json
QA pairs saved to qa_pairs_ocr/dialy/dialy_qa_pairs.json
QA pairs saved to qa_pairs_ocr/dialy/dialy_qa_pairs.json
QA pairs saved to qa_pairs_ocr/dialy/dialy_qa_pairs.json
QA pairs saved to qa_pairs_ocr/dialy/dialy_qa_pairs.json
QA pairs saved to qa_pairs_ocr/dialy/dialy_qa_pairs.json
QA pairs saved to qa_pairs_ocr/dialy/dialy_qa_pairs.json
QA pairs saved to qa_pairs_ocr/dialy/dialy_qa_pairs.json
QA pairs saved to qa_pairs_ocr/dialy/dialy_qa_pairs.json
QA pairs saved to qa_pairs_ocr/dialy/dialy_qa_pairs.json
QA pairs saved to qa_pairs_ocr/dialy/dialy_qa_pairs.json
QA pairs saved to qa_pairs_ocr/dialy/dialy_qa_pairs.json
QA pairs saved to qa_pairs_ocr/dialy/dialy_qa_pairs.json
QA pairs saved to qa_pairs_ocr/dialy/dialy_qa_pairs.json
QA pairs saved to qa_pairs_ocr/